# Working with groups of rows of a data frame

### Bogumił Kamiński

In [1]:
using DataFrames

In [2]:
using CSV

In [3]:
using Arrow

In [4]:
using Statistics

In [5]:
using FreqTables

In [6]:
using Pipe

This part of the tutorial assumes that you have run *3. Working with text files* part so that you have auto2.csv and auto2.arrow files in your working directory.

In [7]:
ENV["LINES"], ENV["COLUMNS"] = 15, 200

(15, 200)

In [8]:
df = CSV.File("auto2.csv") |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String63,String15
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii,plymouth
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina,pontiac


In [9]:
df2 = Arrow.Table("auto2.arrow") |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii,plymouth
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina,pontiac


In [10]:
df == df2

true

In general using Arrow.jl is a preferred way to store your data frames, but note that in order to ensure maximum speed it uses its own `AbstractVector` type:

In [11]:
df2.mpg

392-element Arrow.Primitive{Float64, Vector{Float64}}:
 18.0
 15.0
 18.0
 16.0
 17.0
  ⋮
 27.0
 44.0
 32.0
 28.0
 31.0

To materialize columns into standard `Vector`s just `copy` a data frame:

In [12]:
df3 = copy(df2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii,plymouth
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina,pontiac


In [13]:
df3.mpg

392-element Vector{Float64}:
 18.0
 15.0
 18.0
 16.0
 17.0
  ⋮
 27.0
 44.0
 32.0
 28.0
 31.0

You can find more discussion about using Arrow.jl with DataFrames.jl [here](https://bkamins.github.io/julialang/2020/11/06/arrow.html).

We want to group our data frame by `:brand` column:

In [14]:
gdf = groupby(df, :brand)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String63,String15
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
3,15.0,8.0,400.0,150.0,3761.0,9.5,70.0,1.0,chevrolet monte carlo,chevrolet
4,28.0,4.0,140.0,90.0,2264.0,15.5,71.0,1.0,chevrolet vega 2300,chevrolet
5,17.0,6.0,250.0,100.0,3329.0,15.5,71.0,1.0,chevrolet chevelle malibu,chevrolet
6,14.0,8.0,350.0,165.0,4209.0,12.0,71.0,1.0,chevrolet impala,chevrolet
7,22.0,4.0,140.0,72.0,2408.0,19.0,71.0,1.0,chevrolet vega (sw),chevrolet
8,20.0,4.0,140.0,90.0,2408.0,19.5,72.0,1.0,chevrolet vega,chevrolet
9,13.0,8.0,350.0,165.0,4274.0,12.0,72.0,1.0,chevrolet impala,chevrolet


Note that it is easy (and fast) to find an appropriate group by passing a `Tuple` of values of grouping columns (in our case it is just one column)

In [15]:
gdf[("ford",)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String63,String15
1,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
2,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
3,21.0,6.0,200.0,85.0,2587.0,16.0,70.0,1.0,ford maverick,ford
4,10.0,8.0,360.0,215.0,4615.0,14.0,70.0,1.0,ford f250,ford
5,19.0,6.0,250.0,88.0,3302.0,15.5,71.0,1.0,ford torino 500,ford
6,14.0,8.0,351.0,153.0,4154.0,13.5,71.0,1.0,ford galaxie 500,ford
7,13.0,8.0,400.0,170.0,4746.0,12.0,71.0,1.0,ford country squire (sw),ford
8,18.0,6.0,250.0,88.0,3139.0,14.5,71.0,1.0,ford mustang,ford
9,21.0,4.0,122.0,86.0,2226.0,16.5,72.0,1.0,ford pinto runabout,ford


Using the `combine` function we can easily calculate some aggregates by group:

In [16]:
brand_mpg = combine(gdf, :mpg => mean)

,brand,mpg_mean
,String15,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
6,pontiac,20.0125
7,dodge,22.0607
8,toyota,28.372
9,datsun,31.113


The `:mpg => mean` syntax means that we pass a column `:mpg` to be processed by the `mean` function for each group.

We could have added a target column name to the generated column in the following way:

In [17]:
combine(gdf, :mpg => mean => :mean_mpg)

,brand,mean_mpg
,String15,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
6,pontiac,20.0125
7,dodge,22.0607
8,toyota,28.372
9,datsun,31.113


Now let us see how one can sort a data frame. In order to see alll rows of a data frame we temporarily change the number of rows printed.

In [18]:
ENV["LINES"] = 50

50

In [19]:
sort!(brand_mpg, :mpg_mean, rev=true) # sort! performs an in-place operation

,brand,mpg_mean
,String15,Float64
1,vw,39.0167
2,nissan,36.0
3,triumph,35.0
4,honda,33.7615
5,datsun,31.113
6,mazda,30.86
7,subaru,30.525
8,vokswagen,29.8
9,renault,29.6667


A typical data cleaning task is to check its consistency. In this case it would be making sure that each brand has a unique origin. We will do it in several ways.

First we use FreqTables.jl:

In [20]:
freqtable(df, :brand, :origin)

37×3 Named Matrix{Int64}
 brand ╲ origin │ 1.0  2.0  3.0
────────────────┼──────────────
"amc"           │  27    0    0
"audi"          │   0    7    0
"bmw"           │   0    2    0
"buick"         │  17    0    0
"cadillac"      │   2    0    0
"capri"         │   1    0    0
"chevroelt"     │   1    0    0
"chevrolet"     │  43    0    0
"chevy"         │   3    0    0
"chrysler"      │   6    0    0
"datsun"        │   0    0   23
"dodge"         │  28    0    0
"fiat"          │   0    8    0
"ford"          │  48    0    0
"hi"            │   1    0    0
"honda"         │   0    0   13
"maxda"         │   0    0    2
"mazda"         │   0    0   10
"mercedes"      │   0    1    0
"mercedes-benz" │   0    2    0
"mercury"       │  11    0    0
"nissan"        │   0    0    1
"oldsmobile"    │  10    0    0
"opel"          │   0    4    0
"peugeot"       │   0    8    0
"plymouth"      │  31    0    0
"pontiac"       │  16    0    0
"renault"       │   0    3    0
"saab"         

Now let us create a chain of operations using Pipe.jl. Here `_` denotes the position of the argument passed from the previous step in the chain.

In [21]:
orig_brand = @pipe df |>
                   groupby(_, :brand) |>
                   combine(_, :origin => x -> length(unique(x)))

,brand,origin_function
,String15,Int64
1,chevrolet,1
2,buick,1
3,plymouth,1
4,amc,1
5,ford,1
6,pontiac,1
7,dodge,1
8,toyota,1
9,datsun,1


Let us make sure that we always get exactly one origin:

In [22]:
extrema(orig_brand.origin_function)

(1, 1)

We also could have grouped our data frame by two columns:

In [23]:
origin_brand2 = @pipe df |>
                      groupby(_, [:origin, :brand]) |>
                      combine(_, nrow)

,origin,brand,nrow
,Float64,String15,Int64
1,1.0,chevrolet,43
2,1.0,buick,17
3,1.0,plymouth,31
4,1.0,amc,27
5,1.0,ford,48
6,1.0,pontiac,16
7,1.0,dodge,28
8,1.0,chevy,3
9,1.0,hi,1


Note that passing `nrow` as a combining operation gives us the number of rows per group.

Now we can do the `unstack` operation on the result to get a similar table that we got with `freqtable`:

In [24]:
origin_vs_brand = unstack(origin_brand2, :brand, :origin, :nrow)

,brand,1.0,2.0,3.0
,String15,Int64?,Int64?,Int64?
1,chevrolet,43,missing,missing
2,buick,17,missing,missing
3,plymouth,31,missing,missing
4,amc,27,missing,missing
5,ford,48,missing,missing
6,pontiac,16,missing,missing
7,dodge,28,missing,missing
8,chevy,3,missing,missing
9,hi,1,missing,missing


The only difference is that missing combinations are given as `missing` and `freqtable` produced `0` there. It is easy to fix using broadcasting:

In [25]:
coalesce.(origin_vs_brand, 0)

,brand,1.0,2.0,3.0
,String15,Int64,Int64,Int64
1,chevrolet,43,0,0
2,buick,17,0,0
3,plymouth,31,0,0
4,amc,27,0,0
5,ford,48,0,0
6,pontiac,16,0,0
7,dodge,28,0,0
8,chevy,3,0,0
9,hi,1,0,0


Note that column names of a data frame can be any sequence of characters as in this case:

In [26]:
names(origin_vs_brand)

4-element Vector{String}:
 "brand"
 "1.0"
 "2.0"
 "3.0"

In [27]:
propertynames(origin_vs_brand)

4-element Vector{Symbol}:
 :brand
 Symbol("1.0")
 Symbol("2.0")
 Symbol("3.0")

In such situations it is easiest to refer to such columns using strings, as `Symbol`s are not super easy to write when they have to represent sequences that are not allowed as variable names as you can see above.

In [28]:
origin_vs_brand."1.0"

37-element Vector{Union{Missing, Int64}}:
 43
 17
 31
 27
 48
 16
 28
  3
  1
 11
 10
  6
  1
  1
  2
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing
   missing

Finally let us show that columns of a data frame can hold any data type.

In [29]:
origin_brand3 = @pipe df |>
                      groupby(_, :origin) |>
                      combine(_, :brand => x -> Ref(unique(x)))

,origin,brand_function
,Float64,Array…
1,1.0,"[""chevrolet"", ""buick"", ""plymouth"", ""amc"", ""ford"", ""pontiac"", ""dodge"", ""chevy"", ""hi"", ""mercury"", ""oldsmobile"", ""chrysler"", ""chevroelt"", ""capri"", ""cadillac""]"
2,2.0,"[""volkswagen"", ""peugeot"", ""audi"", ""saab"", ""bmw"", ""opel"", ""fiat"", ""volvo"", ""renault"", ""vw"", ""mercedes-benz"", ""mercedes"", ""vokswagen"", ""triumph""]"
3,3.0,"[""toyota"", ""datsun"", ""mazda"", ""toyouta"", ""maxda"", ""honda"", ""subaru"", ""nissan""]"


Note that we used `Ref` to protect the return value of our function to be broadcasted (this is the same trick as in standard broadcasting in Julia). Otherwise we would get:

In [30]:
@pipe df |> groupby(_, :origin) |> combine(_, :brand => unique)

,origin,brand_unique
,Float64,String15
1,1.0,chevrolet
2,1.0,buick
3,1.0,plymouth
4,1.0,amc
5,1.0,ford
6,1.0,pontiac
7,1.0,dodge
8,1.0,chevy
9,1.0,hi


Now we check if the groups are pairwise disjoint (this is a bit tricky, but it is much fun to be able to write such code in Julia):

In [31]:
for i in axes(origin_brand3, 1)
    for j in i+1:nrow(origin_brand3)
        println("$i vs $j => ", intersect(origin_brand3.brand_function[[i,j]]...))
    end
end

1 vs 2 => String15[]
1 vs 3 => String15[]
2 vs 3 => String15[]


We can also easily flatten the `origin_brand3` data frame so that `brand_function` column is expanded to span multiple rows (the case we have seen above when we omitted writing `Ref` in `combine`)

In [32]:
flatten(origin_brand3, :brand_function)

,origin,brand_function
,Float64,String15
1,1.0,chevrolet
2,1.0,buick
3,1.0,plymouth
4,1.0,amc
5,1.0,ford
6,1.0,pontiac
7,1.0,dodge
8,1.0,chevy
9,1.0,hi
